# Grey Poupon: A Hip-Hop Lyrical Phenomena!

### I recently saw a video on an interesting trend in the field of Hip-Hop music about Grey Poupon - a brand of dijon mustard - having an unusual amount of references in the genre as an allegory for "the idea of status, luxury, and class" [[1]](#citations).

### While a funny fact, I feel we can analyze this further and consider more aspects regarding the usage of the brand in the Hip-Hop music landscape.

#### The questions we will answer are the following:

1. [How many songs mention the brand?](#brand)
2. [Which artists have made references to Grey Poupon? How many times?](#top_artists)
3. [Are there alternative spellings of Grey (i.e. Gray) used as well?](#grey_vs_gray)
4. [What are the top ten words in the lyrics?](#top_words)
5. [Are there permutations of the word "Poupon" in the lyrics and what are they?](#poupon_permutations)
6. [What are the most common nouns and verbs used in song lyrics with Grey Poupon?](#nouns_and_verbs)
7. [Are there any words in the lyrics the artists are rhyming with "Poupon"? Which words and how many?](#poupon_rhymes)

## Preface: First we will collect our song data from genius.com to clean and analyze later on (song titles, artist names and lyric snippets) [[2]](#citations):

In [1]:
import pyautogui, time, pandas as pd, matplotlib.pyplot as plt, csv, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from textblob import TextBlob

In [2]:
# Activate failsafe (moving mouse cursor to upper-left of window will stop program if needed)
pyautogui.FAILSAFE = True

# Initialize WebDriver
driver = webdriver.Chrome()

# Open the URL of the webpage
url = "https://genius.com/search?q=poupon"
driver.get(url)

# Trigger "Show more lyrics" link to bring up search results
link = driver.find_element(By.LINK_TEXT, "Show more lyric matches")
link.click()

In [3]:
pyautogui.hotkey("f12")
time.sleep(5)

page_html = bs(driver.page_source, "html.parser")


In [4]:
# Extract currently loaded search results
def collect_data(data_list, str_list):
    for list_item in data_list:
        str_list.append(str(list_item))

# Trigger webpage to load more results
def load_more_data():
    pyautogui.hotkey('ctrl', 'f')
    pyautogui.write('load_next()')
    time.sleep(2)
    page_html = bs(driver.page_source, "html.parser")
    html_data_list = list(page_html)
    html_data_list_str = str(html_data_list)
    return html_data_list_str, page_html
    

In [5]:
# Run load_more_data manually one time to feed the initial iteration of the while loop
html_data_list_str = load_more_data()
prior_html_list_str = []

In [ ]:
# When end of results is reached, break while loop and continue program
while len(prior_html_list_str) != len(html_data_list_str):
    # Load more results and assign to lyric_poupon_check_list_str
    prior_html_list_str = html_data_list_str
    html_data_list_str, page_html = load_more_data()

In [ ]:
# Collect song titles
song_title = page_html.find_all("div", {"class": "mini_card-title"})
song_title_list = list(song_title)
song_title_list_str = []

# Convert list elements to strings for cleaning later on
collect_data(song_title_list, song_title_list_str)


In [ ]:
# Collect artist names
artist_name = page_html.find_all("div", {"class": "mini_card-subtitle"})
artist_name_list = list(artist_name)
artist_name_list_str = []

# Convert list elements to strings for cleaning later on
collect_data(artist_name_list, artist_name_list_str)



In [ ]:
# Collect lyrics
lyric_values = page_html.find_all("div", {"class": "mini_card-lyric_excerpt"})
time.sleep(3)
lyric_values_list = list(lyric_values)
lyric_values_list_str = []

# Convert list elements to strings for for cleaning later on
collect_data(lyric_values_list, lyric_values_list_str)

In [ ]:
# Close Genius.com web browser window
time.sleep(2)
driver.close()

In [ ]:
# Remove first few rows at top of lists due to unrelated search results
song_title_list_clean = song_title_list_str[17:]
artist_name_list_clean = artist_name_list_str[11:]
lyric_values_list_clean = lyric_values_list_str[6:]


In [ ]:
# Remove leading and trailing html data from song/artist/lyric values
def remove_html_data(data_list, leading_length, clean_list_2):
    for data_string in data_list:
        clean_data = data_string[leading_length:]
        
        # Remove "\n" with space for .index() to find correct position for trailing_length
        clean_data = clean_data.replace("\n", " ")
        clean_data = clean_data.replace("<span>", "")
        clean_data = clean_data.replace("</span>", "")

        trailing_length = clean_data.index("<")
        clean_data = clean_data[:trailing_length]

        clean_list_2.append(clean_data.lower())
        

In [ ]:
song_title_list_clean_2 = []
remove_html_data(song_title_list_clean, 29, song_title_list_clean_2)


In [ ]:
artist_name_list_clean_2 = []
remove_html_data(artist_name_list_clean, 35, artist_name_list_clean_2)


In [ ]:
lyric_values_list_clean_2 = []
remove_html_data(lyric_values_list_clean, 112, lyric_values_list_clean_2)


In [ ]:
# Combine all three columns into one dataframe
lyric_df = pd.DataFrame(list(zip(song_title_list_clean_2, artist_name_list_clean_2, lyric_values_list_clean_2)),
                       columns=["song_title", "artist_name", "lyric_values"])

### Now that we have extracted our data and imported it to a pandas dataframe, we can begin cleaning it for further use later on:

In [ ]:
# Remove expletives and other offensive language from lyric_values listed in a csv file
    # Note: The CSV file is not shared in this project - you may create your own in the following format:
        # Row 1: Header/Disclaimer row
        # Row 2: Word to censor
        # etc.

# Load words to list
expletive_list = []
with open('C:\\Users\\mnbia\\Documents\\grey_poupon_music_lyrics_analysis\\word_censor_list.csv') as csvfile:
    file_reader = csv.reader(csvfile)
    for row in file_reader:
        expletive_list.append(row)

In [ ]:
# Flatten expletive_list from list of lists format
expletive_list_flat = []
for expl_word_1 in expletive_list:
    for sub_expl_word_1 in expl_word_1:
        expletive_list_flat.append(sub_expl_word_1)

# Remove header/disclaimer from first list element
expletive_list_flat = expletive_list_flat[1:]

In [ ]:
# Iterate through lyric_values column and remove any words found from list
for expl_word_2 in expletive_list_flat:
    lyric_df['lyric_values'] = lyric_df['lyric_values'].str.replace(expl_word_2, '')

###  Let's check in with how the dataframe currently looks:

In [ ]:
# Check first and last 5 entries to see how the results look
pd.set_option('max_colwidth', 400)
lyric_df

### Since the last five search results appear to be irrelevant, find how many don't contain "poupon":

In [ ]:
# Search lyric values do not contain "Poupon" in them
lyric_df.lyric_values.str.contains('Poupon', regex=False, case=False).value_counts()

In [ ]:
# Remove songs that do not have "Poupon" in their lyric values
    # Create dataframe that only shows entries whose lyrics contain "Poupon"
poupon_lyric_df = lyric_df[lyric_df['lyric_values'].str.contains('Poupon', regex=False, case=False) == True]
poupon_lyric_df

### It looks like French songs are inflating results due to poupon meaning "doll/baby" as opposed to referencing the mustard brand:


In [ ]:
# Solution is to remove songs identified as being in French
# Method to ID French songs is with .contains() and regex formula to find all values in text column that contain a character other than Basic Latin ASCII encoding
# This will eliminate most songs mentioning "poupon" not related to the mustard brand

# Regex logic:
    # The regex formula allows the program to find accented letters not found in the english alphabet by searching by unicode (i.e. Basic Latin 0000 - 007F)
    # [^...] = any character other than what's in bracket
    # "-" (dash) = range of numerical values between left and right values
    # \x = regex code for period (".") which in turn means "any character" to dynamically search range of unicode digits
    # Resulting translation of regex is "any character that is NOT basic latin"
    # Translation of full pandas statement with regex formula is "filter out any entries that contain characters outside of Basic Latin (i.e. avoid ñ, ó, etc.)"

poupon_lyric_df = poupon_lyric_df[~poupon_lyric_df['lyric_values'].str.contains(r'[^\x00-\x7F]')]

### Now that the dataframe has been filtered down to relevant data, we can now begin our analysis in-depth

In [ ]:
poupon_lyric_df

<a id='brand'></a>
## 1. How many songs mention the brand?

### 1.1 Checking the length of the dataframe, we can see that there are 401 total songs mentioning the word "poupon":

In [ ]:
len(poupon_lyric_df)

<a id='top_artists'></a>
## 2. Which artists have made references to Grey Poupon? How many times?

In [ ]:
# Check top ten artists mentioning "Poupon" in lyrics
poupon_lyric_df['artist_name'].value_counts().head(10)

In [ ]:
# Remove Genius translation artists
poupon_lyric_df = poupon_lyric_df[poupon_lyric_df['artist_name'].str.contains('Genius', regex=False) == False]


In [ ]:
# Top ten artists with songs mentioning "poupon" in lyrics
ax = poupon_lyric_df['artist_name'].value_counts().head(10).plot.bar(title='Top Ten Artists with "Poupon" in Lyrics', color='gold')
ax.set_facecolor('gray')
plt.xticks(rotation=45, ha='right')
ax.set(xlabel=None)

# Add labels to top of each bar
for container in ax.containers:
    ax.bar_label(container, color='w')



<a id='grey_vs_gray'></a>
## 3. Are there alternative spellings of Grey (i.e. Gray) used as well?

### 3.1 First we will make all characters lowercase and remove special characters (i.e. apostrophes, commas, etc.):

In [ ]:
# Grey vs Gray vs neither comparison
    # Share of songs that spell grey, gray or neither in lyrics
    
# Make all characters lowercase in lyric_values to avoid double entries due to case sensitivity
lower_poupon_lyrics = poupon_lyric_df['lyric_values'].str.lower()

In [ ]:
# Remove all non-alphabet characters from lyric_values
for lyric_line in lower_poupon_lyrics:
    for character in lyric_line:
        if character.isalpha() == False:
            lower_poupon_lyrics = lower_poupon_lyrics.str.replace(character, ' ', regex=False)

In [ ]:
# With an "e"
with_e_series = lower_poupon_lyrics[lower_poupon_lyrics.str.contains('grey', regex=False, case=False)] == True

# With an "a"
with_a_series = lower_poupon_lyrics[lower_poupon_lyrics.str.contains('gray', regex=False, case=False)] == True

# Neither word mentioned in lyrics
neither_series = (lower_poupon_lyrics.str.contains('grey', regex=False, case=False) == False) & (lower_poupon_lyrics.str.contains('gray', regex=False, case=False) == False)
neither_series = (neither_series[neither_series] == True)

In [ ]:
e_vs_a_vs_none_df = pd.DataFrame(index=['grey', 'gray', 'neither'], data=[len(with_e_series), len(with_a_series), len(neither_series)], columns=['count'])
e_vs_a_vs_none_df


### 3.2 With our count of grey, gray and neither categories, we now have a pie chart showing the breakdown. It's interesting that there are spellings with an "a" when the brand is with an "e"!

In [ ]:
# Pie chart for grey vs gray vs neither
e_vs_a_vs_none_pie = e_vs_a_vs_none_df.plot.pie(y='count', colors=['gray', 'lightgray', 'gold'], ylabel='', autopct='%.1f%%')
e_vs_a_vs_none_pie.set_title('Grey vs Gray Usage in Lyrics');

<a id='top_words'></a>
## 4. What are the top ten words in the lyrics?

### 4.1 Now we will create split the lyric values into individual words, stack words into one column then count occurrences:

In [ ]:
word_count_df_2 = lower_poupon_lyrics.str.split(expand=True)

word_count_df_3 = word_count_df_2.stack()


In [ ]:
top_words_series = word_count_df_3.value_counts()
top_words_series.head(10)

In [ ]:
# Remove single character words from series as they do not have much value in meaning
for word in top_words_series.index:
    if len(word) < 2:
        top_words_series = top_words_series.drop(labels=word)


In [ ]:
top_words_series.head(10)

### 4.2 Here is the first bar graph using the word data. However, the results are burying verbs, nouns and adjectives, so we will use the textblob library later on to filter only for those.

In [ ]:
# Top ten words in lyrics
    # Results are burying verbs, nouns and adjectives, so we will use the textblob library later on to filter only for those
ax = top_words_series.head(10).plot.bar(title='Top Ten Words in Lyrics', color='gold')
ax.set_facecolor('gray')
plt.xticks(rotation=45, ha='right')
ax.set(xlabel=None)

# Add labels to top of each bar
for container in ax.containers:
    ax.bar_label(container, color='w')


<a id='poupon_permutations'></a>
## 5. Are there permutations of the word "Poupon" in the lyrics and what are they?

### 5.1 Often in the Hip Hop genre, artists take liberties in the grammar, pronounciation and spellings of words. Let's take a look at all words containing "poupon":

In [ ]:
# Create dataframe from series
top_words_df = top_words_series.to_frame()
top_words_df = top_words_df.reset_index()

In [ ]:
top_words_df = top_words_df.rename(columns={'index':'word'})
top_words_df

### 5.2 Just as I hoped, there are quite a few different takes on the word! Something interesting to note is how it was changed to a verb with the spelling of "pouponing".

In [ ]:
# See all permutations of "poupon" (i.e. all words containing "poupon")
top_words_df[top_words_df['word'].str.contains('poupon')]

<a id='nouns_and_verbs'></a>
## 6. What are the most common nouns and verbs used in song lyrics with Grey Poupon?

In [ ]:
# Make a list from words series and combine to one string for use by textblob
word_list = top_words_df['word'].to_list()
word_list_str = ' '.join(word_list)

# Remove "\'" characters from string contents
word_list_str = word_list_str.replace("\'", "")


### 6.1 Due to textblob's function of splitting contraction words into two (don't --> don, 't), we will remove all contraction words from dataset with a list of words from wikipedia [[3]](#citations)

In [ ]:
# Due to textblob's function of splitting contraction words into two (don't --> don, 't),
    # we will remove all contraction words from dataset with a list of words from wikipedia
# Open the URL of the webpage and scrape page's html
url = requests.get('https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions') 

page_html = bs(url.content) 

# Find table of contraction words to scrape in html
table_data = page_html.find_all("tr")
table_data_list = list(table_data)
table_data_list_str = []

In [ ]:
# Convert beautifulsoup object to list of strings
collect_data(table_data_list, table_data_list_str)

# Remove first few rows at top of lists due to unrelated search results
table_data_list_str = table_data_list_str[2:]

In [ ]:
clean_table_data_list = []

# Remove leading and trailing html data from constraint word values
remove_html_data(table_data_list_str, 9, clean_table_data_list)

In [ ]:
# Remove "(informal)", "(poetic)" and "(formal)" from words
clean_table_data_list_2 = []

for word in clean_table_data_list:
    if ' (informal)' in word:
        clean_word = word.replace(' (informal)', '')
        clean_table_data_list_2.append(clean_word)
        
    elif ' (poetic)' in word:
        clean_word = word.replace(' (poetic)', '')
        clean_table_data_list_2.append(clean_word)
        
    elif ' (formal)' in word:
        clean_word = word.replace(' (formal)', '')
        clean_table_data_list_2.append(clean_word)
    
    elif word == '':
        continue
    
    else:
        clean_table_data_list_2.append(word)

# Remove "/", duplicate "n" value and punctuation characters from word entry
clean_table_data_list_3 = []
for word_2 in clean_table_data_list_2:
    if r"/" in word_2:
        clean_word_2 = word[:3]
        clean_word_2 = clean_word_2.replace("’", "")
        clean_table_data_list_3.append(clean_word_2)
    
    elif "’" in word_2:
        clean_word_2 = word_2.replace("’", "")
        clean_table_data_list_3.append(clean_word_2)
        
    elif r"'" in word_2:
        clean_word_2 = word_2.replace(r"'", "")
        clean_table_data_list_3.append(clean_word_2)
        
    else:
        clean_table_data_list_3.append(word_2)



In [ ]:
# Compare word_list_str to list of contraction words
    # if found, word is removed from word_list_str
for contraction_word in clean_table_data_list_3:
    if contraction_word in word_list_str:
        word_list_str = word_list_str.replace(contraction_word, '')


In [ ]:
# Convert back to list for iterating through for textblob tags
#     # Remove all non-alphabet characters
word_list = word_list_str.split()


In [ ]:
# for loop to input text to textblob function for word catagory tagging
word_list_tags = []
word_list_name = []
for word in word_list:
    word_list_tag_blob = TextBlob(word)
    word_list_name_blob = TextBlob(word)
    word_list_name.append(word_list_name_blob.tags[0][0])
    word_list_tags.append(word_list_tag_blob.tags[0][1])


In [ ]:
# Add tags as a new column to top_words_df
word_tag_df = pd.DataFrame(list(zip(word_list_name, word_list_tags)), columns=['word', 'word_tag'])

# Drop duplicates resulting from punctuation cleaning
word_tag_df = word_tag_df.drop_duplicates()
word_tag_df

In [ ]:
# Join dataframes against word values to add tag column to top_words_df
top_words_df_2 = pd.merge(top_words_df, word_tag_df, on='word', how='left')


In [ ]:
# Filter to only show nouns (singular, plural, proper, plural proper)
top_words_df_2_nouns = top_words_df_2[(top_words_df_2['word_tag'] == 'NN') | (top_words_df_2['word_tag'] == 'NNS') | (top_words_df_2['word_tag'] == 'NNP') | (top_words_df_2['word_tag'] == 'NNPS')] 


In [ ]:
# Set word series to index to be usable for bar chart
top_words_df_2_nouns = top_words_df_2_nouns.set_index('word')


In [ ]:
top_words_df_2_nouns.head(10)

In [ ]:
# Drop word_tag series from dataframe
top_words_df_2_nouns = top_words_df_2_nouns.drop(columns=['word_tag'])


In [ ]:
# Remove "poupon", "grey" and "gray" due to higher counts from other nouns by a wide marginS
top_words_df_2_nouns = top_words_df_2_nouns.drop(['poupon', 'grey', 'gray'], axis='index')

### 6.2 Now that we can analyze the dataset for nouns and verbs, we can see for nouns that some are related to food (bread and eat)! Note that while "eat" is usually a verb, it can in fact be a noun in some instances. [[4]](#citations)

In [ ]:
# Top ten nouns in lyrics
ax = top_words_df_2_nouns['count'].head(10).plot.bar(title='Top Ten Nouns in Lyrics', color='gold')
ax.set_facecolor('gray')
plt.xticks(rotation=45, ha='right')
ax.set(xlabel=None)

# Add labels to top of each bar
for container in ax.containers:
    ax.bar_label(container, color='w')

### 6.3 Just like the nouns, there are two related to food (eating and have). This is definitely not a coincidence since we are looking at lyrics surrounding the brand name of a condiment!

In [ ]:
# Filter to only show verbs (singular, plural, proper, plural proper)
top_words_df_2_verbs = top_words_df_2[(top_words_df_2['word_tag'] == 'VB') | (top_words_df_2['word_tag'] == 'VBD') | (top_words_df_2['word_tag'] == 'VBG') | (top_words_df_2['word_tag'] == 'VBN') | (top_words_df_2['word_tag'] == 'VBP') | (top_words_df_2['word_tag'] == 'VBZ')] 


In [ ]:
# Top Ten Verbs in Lyrics
# Set word series to index to be usable for bar chart
top_words_df_2_verbs = top_words_df_2_verbs.set_index('word')
top_words_df_2_verbs.head(10)

In [ ]:
# Top ten verbs in lyrics
ax = top_words_df_2_verbs['count'].head(10).plot.bar(title='Top Ten Verbs in Lyrics', color='gold')
ax.set_facecolor('gray')
plt.xticks(rotation=45, ha='right')
ax.set(xlabel=None)

# Add labels to top of each bar
for container in ax.containers:
    ax.bar_label(container, color='w')

<a id='poupon_rhymes'></a>
## 7. Are there any words in the lyrics that rhyme with "poupon"? Which words and how many?

### 7.1 To begin, we will extract data from rhymezone.com to list words that rhyme with "poupon". [[5]](#citations)

In [ ]:
# Rhyming word comparison
    # Pull list of words that rhyme with "poupon" and see which ones occur in lyrics
# Connect to URL of the webpage and scrape page's html
rhyme_url = requests.get('https://www.rhymezone.com/r/rhyme.cgi?Word=poupon&typeofrhyme=perfect&org1=syl&org2=l&org3=y') 

rhyme_page_html = bs(rhyme_url.content) 

rhyme_list = rhyme_page_html.find_all("a", {"class": "r"})

In [ ]:
# Create series of strings for html lines and remove bottom rows unrelated to rhyme words
rhyme_str_series = pd.Series(rhyme_list).astype(str)
rhyme_str_series = rhyme_str_series[rhyme_str_series.str.contains('moreideas') == False]

In [ ]:
# Remove leading and trailing html data from word values
clean_rhyme_word_list = []
for clean_rhyme_word in rhyme_str_series:
    # Get index position of ">" to slice leading html
    clean_rhyme_word_lead_index = clean_rhyme_word.index('>')
    clean_rhyme_word = clean_rhyme_word[clean_rhyme_word_lead_index + 1:]

    # Get index position of "<" to slice trailing html
    clean_rhyme_word_trail_index = clean_rhyme_word.index('<')
    clean_rhyme_word = clean_rhyme_word[:clean_rhyme_word_trail_index]
    
    clean_rhyme_word_list.append(clean_rhyme_word)

In [ ]:
# Create new series with cleaned (html-free) words
clean_rhyme_str_series = pd.Series(data=clean_rhyme_word_list)

In [ ]:
# Make series of all words from prior dataframe for comparison to rhyme word list
top_words_series = pd.Series(top_words_df_2['word'])
top_words_series.head(10)

In [ ]:
# Compare word_list_str to list of contraction words
    # if found, word is removed from word_list_str
lyric_rhyme_word_list = []
for rhyme_word in clean_rhyme_str_series:
    for lyric_word in top_words_series:
        if rhyme_word == lyric_word:
            lyric_rhyme_word_list.append(rhyme_word)

In [ ]:
lyric_rhyme_word_series = pd.Series(data=lyric_rhyme_word_list)

In [ ]:
lyric_rhyme_word_df = top_words_df_2[top_words_df_2['word'].isin(lyric_rhyme_word_series)]
lyric_rhyme_word_df = lyric_rhyme_word_df.drop(columns=['word_tag'])
lyric_rhyme_word_df = lyric_rhyme_word_df.set_index('word')

### 7.2 Now we can see the creativity in the lyrics: Not only are there a high amount of words that rhyme with "poupon", but some of them even have a related meaning (such as "dijon", which is the style of mustard).

In [ ]:
# Top ten words that rhyme with poupon in lyrics
ax = lyric_rhyme_word_df['count'].head(10).plot.bar(title='Top Ten Poupon Rhymes in Lyrics', color='gold')
ax.set_facecolor('gray')
plt.xticks(rotation=45, ha='right')
ax.set(xlabel=None)

# Add labels to top of each bar
for container in ax.containers:
    ax.bar_label(container, color='w')

<a id='citations'></a>
# Citations:
1. Caswell, Estelle and Sarah Frostenson. “How Grey Poupon Became Hip-Hop’s Favorite Condiment.” Vox, 12 Oct. 2016, www.vox.com/videos/2016/10/12/13250360/grey-poupon-in-hip-hop
<br>
<br>
2. “Song Lyrics & Knowledge.” Genius, genius.com/search?q=poupon. Accessed 23 May 2024. 
<br>
<br>
3. “List of English Contractions.” Wikipedia, Wikimedia Foundation, 18 May 2024, en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions.
<br>
<br>
4. “Eat Definition & Meaning.” Merriam-Webster, Merriam-Webster, www.merriam-webster.com/dictionary/eat#dictionary-entry-2. Accessed 23 May 2024.
<br>
<br>
5. “Poupon Rhymes.” RhymeZone, www.rhymezone.com/r/rhyme.cgi?Word=poupon&typeofrhyme=perfect&org1=syl&org2=l&org3=y. Accessed 23 May 2024. 


# Future Improvements:

### While we were able to find many words that rhyme with "poupon", it did not capture them all due to not being in the source to compare the lyrics to. One example is "crouton" a missed opportunity as it is used multiple times and is related to food like Grey Poupon.

In [ ]:
# Filter for "crouton" 
top_words_df_2[top_words_df_2['word'].str.contains('crouton')]

### Solution is to pull rhyming words from additional sources (i.e. other websites) and combine to make the rhyme analysis more thorough.

# Special thanks to Estelle Caswell and Sarah Frostenson from Vox.com for their insightful video and article that inspired this analysis!